#### Adding Nodes
- You’ve got the skeleton (the StateGraph), now it’s time to give it some brains. 
- Adding nodes is your next step.
- In LangGraph, a node is essentially a function that takes the current State, does some work (like calling an LLM or searching a database), and returns an updated version of that State.
- 

##### A Node Function
-  A node function can have 3 patterns:
```
def my_node(state: MyState)
def my_node(state: MyState, config: RunnableConfig):
def my_node(state: MyState, config: RunnableConfig, *, store: BaseStore):
```
- **state** - A positional argument, that must match the class or TypedDict you passed into the StateGraph constructor.
The Variable Name it is conventionally called **state**.
- **config**: RunnableConfig (type is imported from langchain_core.runnables).  
If you include it, it must come after state.  
You only include this if you need to access the "metadata" of the run (like a thread_id) inside the function body.  
If you include it, it must come after state.  
- **store** - A keyword only store: BaseStore   (so you must use the * sign to make it keyword only)
Type must be imported from langgraph.store.base  
LangGraph will only inject the store if it sees a parameter explicitly named store defined as keyword-only.


##### Node Function with just state

In [4]:
# Example: node function with just state
from typing import TypedDict, List

# 1. Define your State class
class MyState(TypedDict):
    messages: List[str]
    category: str  # This is what our node will update

# 2. Define the Node Function
def intent_classifier(state: MyState):
    """
    A pure node: It only needs the current state to function.
    It inspects the last message and returns a state update.
    """
    last_message = state["messages"][-1].lower()
    
    if "billing" in last_message or "payment" in last_message:
        return {"category": "billing"}
    
    if "help" in last_message or "support" in last_message:
        return {"category": "technical_support"}
    
    return {"category": "general_inquiry"}

##### Node function with state and config
- A node function using the 2-parameter pattern (state and config).
- It makes the most sense when your logic depends on who is running the graph or how it was configured
- The most common real-world use case is fetching user-specific settings or API keys that are stored in the configurable field of the RunnableConfig.

In [5]:
from typing import TypedDict
from langchain_core.runnables import RunnableConfig

class MyState(TypedDict):
    greeting: str

def fetch_user_profile_node(state: MyState, config: RunnableConfig):
    """
    Pattern 2: Uses State (to see current data) 
    AND Config (to identify the specific user context).
    """
    # 1. Access the 'configurable' dict inside the config
    # This is where you pass 'out-of-band' data like user IDs
    user_id = config["configurable"].get("user_id", "anonymous")
    
    # 2. Logic based on that config (e.g., a mock database lookup)
    user_db = {"user_123": "Alice", "user_456": "Bob"}
    user_name = user_db.get(user_id, "Guest")
    
    # 3. Return the state update
    return {"greeting": f"Hello, {user_name}! How can I help you today?"}

##### Node function with 3 parameters
- The 3rd pattern (using the store) is the most advanced. 
- It allows a node to act like a librarian: it can read from the current State, use the Config to know whose file to look for, and use the Store to pull a "memory" from a completely different conversation.
- In this example, we use the store to see if we've ever learned the user's name in any previous thread.

In [6]:
from typing import TypedDict
from langchain_core.runnables import RunnableConfig
from langgraph.store.base import BaseStore

class MyState(TypedDict):
    messages: list[str]
    user_name: str

def recall_user_node(state: MyState, config: RunnableConfig, *, store: BaseStore):
    """
    Pattern 3: State + Config + Store.
    We use config to get the user_id, then check the store for long-term memory.
    """
    # 1. Identify the user (from Config)
    user_id = config["configurable"].get("user_id")
    namespace = ("users", user_id) # The "folder" in the store
    
    # 2. Look in the "Filing Cabinet" (Store)
    # This data persists even if the current 'state' is brand new
    memory = store.get(namespace, "profile")
    
    if memory:
        name = memory.value.get("name")
        return {"user_name": name, "messages": [f"Welcome back, {name}!"]}
    
    return {"messages": ["Welcome! I don't believe we've met. What is your name?"]}